Create a function to measure a financial health of a company 

In [30]:
import OpenDartReader 
import sys, os
sys.path.append(os.path.dirname(os.getcwd()))  
from tools.dictionary import ACCOUNT_NAME_DICTIONARY, BS_ACCOUNTS, IS_ACCOUNTS, DART_APIS, MODIFIED_REPORT
import pandas as pd
import datetime, time

def collect_financial_reports(dart, code, duration=None): # duration as years

    def sj_div(account_nm):
        if account_nm in BS_ACCOUNTS:
            return 'BS'
        elif account_nm in IS_ACCOUNTS:
            return 'IS'
        else:
            raise Exception('No BS, IS account exception')

    def post_process(rec): 
        if len(rec) > 0:
            # in some cases, certain data is '-'
            rec.replace('-', pd.NA, inplace=True)
            # add more logic for post_process if needed
            # ...
            return rec
        else: 
            return rec

    def get_prev_quarter(yr, qtr): # qtr in [1, 2, 3, 4]
        if qtr == 1: 
            return yr-1, 4 
        else: 
            return yr, qtr-1 

    def get_prev_quarter_except_FY(yr, qtr): # qtr in [1, 2, 3, 4]
        if qtr == 1:
            return yr-1, 3 
        else: 
            return yr, qtr-1 

    # find initial report
    year = datetime.datetime.now().year
    month = datetime.datetime.now().month
    quarter = month//3 + 1

    reprt_code_dict = {1: '11013', 2: '11012', 3: '11014', 4: '11011'}

    y = year
    q = quarter
    ind = 0

    while True:
        ind += 1
        y, q = get_prev_quarter(y, q)
        rec = dart.finstate(code, y, reprt_code=reprt_code_dict[q])
        if len(rec) > 0:
            break
        if ind == 8: # search initial data within last 8 quarters
            # raise Exception('Data not available for code:', code) 
            return pd.DataFrame(), 'Data Not Available'

    record = pd.DataFrame(columns=['stock_code', 'fs_div', 'sj_div', 'account_nm', 'date_updated'])
    accounts = BS_ACCOUNTS + IS_ACCOUNTS
    date_updated = datetime.datetime.today().strftime('%Y-%m-%d')

    for i in range(len(accounts)):
        record.loc[i] = [code, 'CFS', sj_div(accounts[i]), accounts[i], date_updated]  
    for i in range(len(accounts)):
        record.loc[i+len(accounts)] = [code, 'OFS', sj_div(accounts[i]), accounts[i], date_updated]  

    if rec['currency'][0] != 'KRW':
        # raise Exception('Currency is not in KRW for code: ', code)
        return pd.DataFrame(), 'Currency Not in KRW'

    y_init = y
    q_init = q

    # data collection method
    # Step 1. collect annual data (and for the last report, collect y-1 and y-2 data)
    # Step 2. collect quarterly data (and if data for the same quarter in the last year), and add 4Q quarterly data column if possible (i.e., if full year data is available) 
    #     note that for BS items, no need to collect previous year data in quarterly data collecting (not provided in Dart)

    # Step 1:
    if q_init != 4: 
        y = y_init-1
    rec = dart.finstate(code, y, reprt_code=reprt_code_dict[4])
    rec = post_process(rec)

    if duration == None or duration <= 0:
        duration = 10000 # a large number enough

    ind = 0
    while len(rec)>0:
        data_term = 'FY'+str(y)
        # print(data_term)
        rec[data_term] = rec['thstrm_amount'].str.replace(',','').astype('Int64')
        record = pd.merge(record, rec[['stock_code', 'fs_div', 'account_nm', data_term]], how='left', left_on=['stock_code', 'fs_div', 'account_nm'], right_on=['stock_code', 'fs_div', 'account_nm'])

        ind += 1
        if ind == duration:
            break

        # check if there is data in prev year 
        prev_year_rec = dart.finstate(code, y-1, reprt_code=reprt_code_dict[4]) 
        prev_year_rec = post_process(prev_year_rec)
        if len(prev_year_rec) == 0: 
            prev_term = 'FY'+str(y-1)
            pprev_term = 'FY'+str(y-2)
            rec[prev_term] = rec['frmtrm_amount'].str.replace(',','').astype('Int64')
            rec[pprev_term] = rec['bfefrmtrm_amount'].str.replace(',','').astype('Int64')
            record = pd.merge(record, rec[['stock_code', 'fs_div', 'account_nm', prev_term, pprev_term]], how='left', left_on=['stock_code', 'fs_div', 'account_nm'], right_on=['stock_code', 'fs_div', 'account_nm'])
            break
        else: 
            y = y-1
            rec = prev_year_rec

    # Step 2:
    if q_init == 4: 
        y = y_init
        q = 3
    else: 
        y = y_init
        q = q_init
    rec = dart.finstate(code, y, reprt_code=reprt_code_dict[q])
    rec = post_process(rec)

    ind = 0
    while len(rec)>0:
        data_term = str(y)+'_'+str(q)+'Q'
        # print(data_term)
        rec[data_term] = rec['thstrm_amount'].str.replace(',','').astype('Int64')
        record = pd.merge(record, rec[['stock_code', 'fs_div', 'account_nm', data_term]], how='left', left_on=['stock_code', 'fs_div', 'account_nm'], right_on=['stock_code', 'fs_div', 'account_nm'])

        # adding 4Q data if 'thstr_add_amount' is available
        # NOTE:
        # some sj_div items are neither IS or BS, and None... 
        # which leaves, 4Q data as sum of 1-3Q... 
        # you may consider correct this in the future
        if q == 3 and 'thstrm_add_amount' in rec.columns:
            if 'FY'+str(y) in record.columns:
                q4_term = str(y)+'_4Q'
                rec.loc[rec['sj_div']=='IS', q4_term] = rec.loc[rec['sj_div']=='IS','thstrm_add_amount'].str.replace(',','').astype('Int64')
                record = pd.merge(record, rec[['stock_code', 'fs_div', 'account_nm', q4_term]], how='left', left_on=['stock_code', 'fs_div', 'account_nm'], right_on=['stock_code', 'fs_div', 'account_nm'])
                record.loc[record['sj_div']=='IS', q4_term] = record.loc[record['sj_div']=='IS','FY'+str(y)]-record.loc[record['sj_div']=='IS', q4_term]
                record.loc[record['sj_div']=='BS', q4_term] = record.loc[record['sj_div']=='BS','FY'+str(y)]

        ind += 1
        if ind == duration*3:
            break

        # check if there is data in the prev year, the same quarter
        prev_year_rec = dart.finstate(code, y-1, reprt_code=reprt_code_dict[q]) 
        prev_year_rec = post_process(prev_year_rec)
        if len(prev_year_rec) == 0 and 'frmtrm_amount' in rec.columns: 
            prev_term = str(y-1)+'_'+str(q)+'Q'
            # add only IS data
            rec.loc[rec['sj_div']=='IS', prev_term] = rec.loc[rec['sj_div']=='IS', 'frmtrm_amount'].str.replace(',','').astype('Int64')
            record = pd.merge(record, rec[['stock_code', 'fs_div', 'account_nm', prev_term]], how='left', left_on=['stock_code', 'fs_div', 'account_nm'], right_on=['stock_code', 'fs_div', 'account_nm'])

            if q == 3 and 'frmtrm_add_amount' in rec.columns:
                if 'FY'+str(y-1) in record.columns:
                    last_q4_term = str(y-1)+'_4Q'
                    rec.loc[rec['sj_div']=='IS', last_q4_term] = rec.loc[rec['sj_div']=='IS','frmtrm_add_amount'].str.replace(',','').astype('Int64')
                    record = pd.merge(record, rec[['stock_code', 'fs_div', 'account_nm', last_q4_term]], how='left', left_on=['stock_code', 'fs_div', 'account_nm'], right_on=['stock_code', 'fs_div', 'account_nm'])
                    record.loc[record['sj_div']=='IS', last_q4_term] = record.loc[record['sj_div']=='IS','FY'+str(y-1)]-record.loc[record['sj_div']=='IS', last_q4_term]
                    record.loc[record['sj_div']=='BS', last_q4_term] = record.loc[record['sj_div']=='BS','FY'+str(y-1)]

        y, q = get_prev_quarter_except_FY(y, q)
        rec = dart.finstate(code, y, reprt_code=reprt_code_dict[q])
        rec = post_process(rec)
    
    # post process
    record.rename(columns={'stock_code':'code', }, inplace=True)
    record['account'] = record['account_nm'].apply(lambda x: ACCOUNT_NAME_DICTIONARY[x] if x in ACCOUNT_NAME_DICTIONARY.keys() else x)

    message = 'success'
    return record, message

def generate_financial_reports_set(sector, duration, log_file, save_file_name=None):
    dart_ind = 0
    dart = OpenDartReader(DART_APIS[dart_ind])

    financial_reports = pd.DataFrame()

    with open(log_file, 'a') as f:
        f.write('Financial data collection log\n')

    error_trial = 0
    error_trial_limit = 10
    sleep_time = 5

    for ix, code in enumerate(sector):
        try:
            current_progress = str(datetime.datetime.now()) + ', no: ' + str(ix) + ', code ' + code+' in process' # / '+df_krx['Name'][code]
            print(current_progress)
            with open(log_file, 'a') as f:
                f.write(current_progress+'\n')

            if dart.find_corp_code(code) == None: 
                current_progress = '----> no: ' + str(ix) + ', code ' + code+' not in corp_code, and therefore data not available' # / '+df_krx['Name'][code]
                print(current_progress)
                with open(log_file, 'a') as f:
                    f.write(current_progress+'\n')
                current_target_indicator += 1
                continue
    
            record, message = collect_financial_reports(dart, code, duration)
            if message == 'success':
                financial_reports = pd.concat([financial_reports, record], ignore_index=True)
                if save_file_name != None:
                    financial_reports.to_feather(save_file_name)
            elif message == 'Data Not Available':
                current_progress = '----> no: ' + str(ix) + ', code ' + code+' data not available, could be a financial institution' # / '+df_krx['Name'][code]
                print(current_progress)
                with open(log_file, 'a') as f:
                    f.write(current_progress+'\n')
            elif message == 'Currency Not in KRW':
                current_progress = '----> no: ' + str(ix) + ', code ' + code+' currency not in KRW, skipping' # / '+df_krx['Name'][code]
                print(current_progress)
                with open(log_file, 'a') as f:
                    f.write(current_progress+'\n')
            else:
                raise Exception('ERROR in execution loop')

            time.sleep(sleep_time)
            error_trial = 0 # reset

        except Exception as e:
            if error_trial < error_trial_limit:
                error_trial += 1
                dart_ind += 1
                dart = OpenDartReader(DART_APIS[dart_ind%3])

            else:
                raise Exception('ERROR TRIAL LIMIT REACHED - Entire Process Halted')
                # break

            current_progress = '----> no: ' + str(ix) + ', code ' + code+' unknown exception; process suspended and to be re-tried' # / '+df_krx['Name'][code]
            print(current_progress)
            print(e)
            with open(log_file, 'a') as f:
                f.write(current_progress+'\n')

            time.sleep(sleep_time*error_trial)

    static_columns = ['code', 'fs_div', 'sj_div', 'account_nm', 'account', 'date_updated']
    financial_reports = pd.concat([financial_reports[static_columns], financial_reports[financial_reports.columns.difference(static_columns)].sort_index(axis=1)], axis=1)
        
    return financial_reports


In [32]:
log_file = 'data/data_collection_log.txt'

today = datetime.datetime.today().strftime('%Y-%m-%d')
yes_day = (datetime.datetime.today() - datetime.timedelta(days = 2)).strftime('%Y-%m-%d')
dart = OpenDartReader(DART_APIS[0])
ls = dart.list(start=yes_day, end=today, kind='A')
display(ls)

full_rescan_code = ls.loc[ls['report_nm'].str.contains(MODIFIED_REPORT)]['stock_code'].values
partial_rescan_code = ls.loc[~ls['report_nm'].str.contains(MODIFIED_REPORT)]['stock_code'].values

db_f = generate_financial_reports_set(full_rescan_code, None, log_file, None)
db_p = generate_financial_reports_set(partial_rescan_code, 1, log_file, None) # 1 year

update_db = pd.concat([db_f, db_p], ignore_index=True)

# NEED TO SORT COLUMNS..... maybe before saving.... 
# NEED TO SORT COLUMNS..... maybe before saving.... 
# NEED TO SORT COLUMNS..... maybe before saving.... 
# NEED TO SORT COLUMNS..... maybe before saving.... 

display(update_db)

,corp_code,corp_name,stock_code,corp_cls,report_nm,rcept_no,flr_nm,rcept_dt,rm
0,00604028,피코그램,376180,K,[기재정정]반기보고서 (2023.06),20231103000073,피코그램,20231103,
1,01215618,와이엠텍,273640,K,분기보고서 (2023.09),20231103000045,와이엠텍,20231103,
2,00370200,와이오엠,066430,K,[기재정정]반기보고서 (2023.06),20231102000039,와이오엠,20231102,
3,01207716,앱코,129890,K,분기보고서 (2023.09),20231102000033,앱코,20231102,


2023-11-03 14:16:43.482740, no: 0, code 376180 in process
2023-11-03 14:16:56.863006, no: 1, code 066430 in process
2023-11-03 14:17:31.650499, no: 0, code 273640 in process
2023-11-03 14:17:40.213710, no: 1, code 129890 in process


,code,fs_div,sj_div,account_nm,account,date_updated,2015_1Q,2015_2Q,2015_3Q,2015_4Q,...,FY2014,FY2015,FY2016,FY2017,FY2018,FY2019,FY2020,FY2021,FY2022,2023_3Q
0,376180,CFS,BS,유동자산,liquid_assets,2023-11-03,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,5673816614,9089337652,21410562926,23695426319,<NA>
1,376180,CFS,BS,비유동자산,illiquid_assets,2023-11-03,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,30321172335,28540434164,28833834586,27888580770,<NA>
2,376180,CFS,BS,자산총계,assets,2023-11-03,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,35994988949,37629771816,50244397512,51584007089,<NA>
3,376180,CFS,BS,유동부채,liquid_debts,2023-11-03,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,6978409819,6524116240,22797801665,20243747160,<NA>
4,376180,CFS,BS,비유동부채,illiquid_debts,2023-11-03,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,19068598507,19426389203,168535604,64016113,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,129890,OFS,BS,자본총계,equity,2023-11-03,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,56138926685,57309619423
100,129890,OFS,IS,매출액,revenue,2023-11-03,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,82663362781,21547884340
101,129890,OFS,IS,영업이익,operating_income,2023-11-03,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-15391295826,1868579072
102,129890,OFS,IS,법인세차감전 순이익,profit_before_tax,2023-11-03,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,-16838441548,1876360196


In [24]:
a = financial_reports_a
b = financial_reports_b
db_ = pd.read_feather('data/financial_reports_main.feather')
display(a.iloc[[0]])
ix = db_.loc[(db_['code']==a.iloc[0]['code']) & (db_['fs_div']==a.iloc[0]['fs_div']) & (db_['account']==a.iloc[0]['account'])].index
display(db_.iloc[ix])
# financial_reports.loc[(financial_reports['code']=='066430')&(financial_reports['fs_div'] == 'CFS')].index

,code,fs_div,sj_div,account_nm,account,date_updated,2015_1Q,2015_2Q,2015_3Q,2015_4Q,...,FY2013,FY2014,FY2015,FY2016,FY2017,FY2018,FY2019,FY2020,FY2021,FY2022
0,376180,CFS,BS,유동자산,liquid_assets,2023-11-03,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,5673816614,9089337652,21410562926,23695426319


,code,fs_div,sj_div,account_nm,account,date_updated,2015_1Q,2015_2Q,2015_3Q,2015_4Q,...,FY2013,FY2014,FY2015,FY2016,FY2017,FY2018,FY2019,FY2020,FY2021,FY2022
34648,376180,CFS,BS,유동자산,liquid_assets,2023-10-05,<NA>,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,5673816614,9089337652,21410562926,23695426319
